---
title: 1.2 Solving Linear Systems via Gaussian Elimination
subject:  Linear Algebraic Systems
subtitle: Forward elimination and back substitution are all you need
short_title: 1.2 Gaussian Elimination
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: systems of linear equations, solution sets, forward elimination, back substitution
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
downloads:
  - file: lecture_notes/Lecture 01 - Systems of linear equations, vectors, matrices, Gauss Elimination and LU-factorization.pdf
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/00_Ch_1_Linear_Algebraic_Systems/021-linsys-gauss.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 01 - Systems of linear equations, vectors, matrices, Gauss Elimination and LU-factorization.pdf>`


## Reading
Material related to this page, as well as additional exercises, can be found in ALA Ch. 1.1, LAA Ch 1.1, and ILA Ch. 2.2.  These notes are mostly based on ALA Ch 1.1.

## Learning Objectives

By the end of this page, you should know:
- what a linear equation is, how to identify one, and how they combine to form a system of linear equations (also called a linear system)
- what a solution set is, and the kinds of solution sets a linear system can have
- how to solve _most_ small systems of linear equations by hand using _Gaussian Elimination_

## Terminology
A _linear equation_[^linear] in the _variables_ $x_1,\dots,x_n$ is an equation that can be written in the form
```{math}
:label: lin-eq
a_1x_1 + a_2x_2+\cdots+a_nx_n = b,
```
where $b$ and the _coefficients_ $a_1,a_2,\dots,a_n$ are real or complex numbers[^real] that are typically known in advance.  The subscript $n$ is a positive integer, and encodes how many variables appear in the linear equation.  When we ask you to solve problems by hand, $n$ is typically 2 or 3, but we will see and solve problems where $n$ is in the hundreds or even thousands!

````{exercise} Linear or not?
:label: ex-linornot
Which of the following equations are linear? 
1. $4x_1 - 5x_2 + 2 = x_1$
2. $x_2 = 2(\pi^2 - x_1)+\sqrt{3}x_3$
3. $4x_1 - 5x_2 = x_1x_2$
4. $x_2 = 2(\pi^2 - x_1)+\sqrt{x_3}$

Explain why or why not.

```{solution} ex-linornot
:class: dropdown
1. Yes because it can be rearranged algebraically to match [](#lin-eq)
2. Yes because it can be rearranged algebraically to match [](#lin-eq)
3. No because of the $x_1x_2$ term
4. No because of the $\sqrt{x_3}$ term.
```
````
[^linear]: We'll see later exactly why we call these equations linear, and the deep implications that linearity has on science and engineering.  For now, it's enough to think of an equation as being linear if the unknowns only appear to the first power, and there are no product terms like $x_1x_2$ or $x_1x_2x_3$, in it.
[^real]: Almost all equations that we will encounter in this class will be defined in terms of real numbers.  In certain settings, we will need to use complex numbers, but don't worry, we'll go over how to work with complex numbers carefully then.

A _system of linear equations_ (or _linear system_) is a collection of one or more [linear equations](#lin-eq) involving the same variables, say $x_1,\dots,x_n$.  An example is
\begin{eqnarray}
\label{simple-linsys00}
x_1+2x_2+x_3 & = 2,\\
2x_1+6x_2+x_3 & =7,\\
x_1+x_2+4x_3 & =3,
\end{eqnarray}
which is a system of three equations in three unknowns.

A _solution_ of the system is a list $(s_1,s_2,\dots,s_n)$ of numbers that makes each equation a true statement when the values $s_1,\dots,s_n$ are substituted for $x_1,\dots,x_n$, respectively.  For instance, $(-3,2,1)$ is a solution of system [](#simple-linsys00) because, when these values are substituted in [](#simple-linsys00) for $x_1,x_2,x_3$, respectively, the equations simplify to $2=2$, $7=7$, and $3=3$.  The set of all possible solutions is called the _solution set_ of the linear system.  We state an important fact regarding the solution set of linear systems that we will understand very deeply by the end of this chapter:

:::{important} Solution Set for Linear Systems
A system of linear equations has
1. no solution, or
2. exactly one solution, or
3. infinitely many solutions.
:::

:::{warning} Applies to Linear Systems Only!
While the three options above are exhaustive for linear systems, they are absolutely not for nonlinear systems of equations!  A familiar example that you would have seen in high school is the solution set to a <wiki:Quadratic_equation>, which can have zero, one, or two solutions!
:::

## Solving Linear Systems
We start with a simple observation: we know how to solve equations that look like
$$
\frac{5}{2}x_1 = 6, \ 5-x_3 = 12, \ 1 + 3x_2 = -4.
$$
These equations are ``easy'' to solve because they only involved one unknown, that we can solve for using simple algebra.  Our strategy will be to develop a systematic way of reducing linear systems, like the example [](#simple-linsys00) above, into an _equivalent system of equations_ that are easy to solve.


To illustrate, let's start with the system of three linear equations [](#simple-linsys00) we introduced above:
\begin{eqnarray}
\label{simple-linsys}
x_1+2x_2+x_3 & = 2,\\
2x_1+6x_2+x_3 & =7,\\
x_1+x_2+4x_3 & =3,
\end{eqnarray}


Our strategy will be to systematically employ the following _very useful_ observation:

:::{prf:observation} Linear System Operation \#1
:label:linop1
Adding multiples of one equation to another equation does not change the solution set
:::

Before continuing, you might try to convince yourself that this observation is true. Our goal is to apply [](#linop1) judiciously to transform the system of linear equations [](#simple-linsys) into a much simpler one that is easy to solve but still has the same solutions as the original.  Any linear system that is derived from the original system by successive application of such operations will be called an _equivalent system_.  An important property is that _equivalent systems have the same solutions._

Our strategy will be to successively eliminate variables in our equations in order of appearance.  So, here, our first step is to eliminate the first variable, $x_1$, from the second equation.  We'll do that by subtracting twice the first equation from the second:
$$
\begin{array}{lrl}
\text{[second equation]} & 2x_1+6x_2+x_3 & =7 \\
\text{-2[first equation]} & -2\left[x_1+2x_2+x_3 \right. & \left. = 2\right]\\
\hline \\
\text{[updated equation]} & 0x_1 + 2x_2 - x_3 & = 3,
\end{array}
$$
so that now, our equivalent system of linear equations is given by
\begin{eqnarray}
\label{simple-linsys0}
x_1+2x_2+x_3 & = 2,\\
2x_2-x_3 & =3,\\
x_1+x_2+4x_3 & =3.
\end{eqnarray}
This system of equations is simpler than [](#simple-linsys) because $x_1$ no longer appears in the second equation.  We can eliminate $x_1$ from the third equation by subtracting the first equation from it, giving 
\begin{eqnarray}
\label{simple-linsys1}
x_1+2x_2+x_3 & = 2,\\
2x_2-x_3 & =3,\\
-x_2+3x_3 & =1.
\end{eqnarray}



The equivalent system [](#simple-linsys1) is even simpler than the original [](#simple-linsys): notice that the second and third equations do not involve $x_1$ (by design), and so constitute a system of two linear equations in two unknowns.  Moreover, once we have solved this subsystem for $x_2$ and $x_3$, we can substitute the answer into the first equation, and we need only solve a single linear equation for $x_1$.

We continue on, with our goal in this next phase to eliminate the second variable, $x_2$, from the third equation by adding $1/2$ the second equation to it.  The result is
\begin{eqnarray}
\label{simple-linsys2}
x_1+2x_2+x_3 & = 2,\\
2x_2-x_3 & =3,\\
\frac{5}{2}x_3 & =\frac{5}{2},
\end{eqnarray}
which is the simple system we have been working towards.  It is in what is called _triangular form_: the first equation depends on all three variables, the second only depends on the second and third variables, and the last equation involves only the last variable.  The process we went through of transforming the original system [](#simple-linsys) to the triangular system [](#simple-linsys2) is an example of _Forward Elimination_: we work our way down the equations, eliminating variables as we go.  Once a system is in triangular form, it can be straightforwardly solved by the method of _Back Substitution_.  As the name suggests, we work backwards, solving the last equation first, which requires that $x_3=1$.  We substitute the result back into the middle equation, which becomes $2x_2-1=3$, with solution $x_2=2$.  We finally substitute the two values $x_2=2$ and $x_3=1$ into the first equation, which becomes $x_1+5=2$, and so the solution to the triangular system [](#simple-linsys2) is
$$
x_1=-3, \quad x_2=2, \quad x_3=1.
$$

Even more exciting is that recalling [](#linop1), we know that the triangular system [](#simple-linsys2) is equivalent to our original system [](#simple-linsys), which means that this is also the solution to our original system of equations (you can check).  We note that in this case, system [](#simple-linsys) has a _unique_, meaning one and only one, solution.  We'll understand why this is the case later in the semester.

This process was incredibly simple and intuitive: add equations together to eliminate ``downstairs'' variables until you have found an equivalent triangular system, which can then be readily solved via back substitution.  Amazingly, barring a few minor complications that can come up from time to time, this is essentially all there is to _Gaussian Elimination_.  This incredibly simple algorithm is also unbelievably important and powerful.  After a brief detour to remind ourselves about matrix and vector notation, we'll revisit Gaussian Elimination through the lens of _matrix factorization_, which will allow us to easily automate and scale out these ideas to systems with hundreds or even thousands of variables. 

## Worked Examples

````{exercise} Finding coefficients
:label: ex1
Find the coefficients $(a, b, c)$ for the set of equations 
\begin{eqnarray}
\label{ex1_sys}
ax + by + cz & = 3,\\
ax + y + cz & =2,\\
x - by - cz & =1,
\end{eqnarray}
so that a solution is $x = -1, y = 1, z = 2$.
:::{hint} Click me for a hint!
:class: dropdown
Identify the unknowns and the set of equations.

:::
```{solution} ex1
:class: dropdown
Substituting $x = -1, y = 1, z = 2$ in [](#ex1_sys), we get the set of equations
\begin{eqnarray}
\label{ex1_soln_1}
-a + b + 2c &= 3,\\
-a + 1 + 2c &=2 \Rightarrow -a + 2c &= 1,\\
-1 - b - 2c &=1 \Rightarrow -b - 2c &= 0,
\end{eqnarray}
The unknowns are $(a, b, c)$ in [](#ex1_soln_1). We use the forward elimination method to first eliminate $a$ by subtracting the first equation from the second equation to get
\begin{eqnarray}
\label{ex1_soln_2}
-a + b + 2c &= 3, \\
-b &= -2,\\
- b - 2c &= 0,
\end{eqnarray}
From [](#ex1_soln_2), we already have a triangular system which is clear after rearranging as given below
\begin{eqnarray}
\label{ex1_soln_3}
-a + 2c + b &= 3, \\
- 2c - b &= 0,\\
-b &= -2,
\end{eqnarray}
Now, using back substitution, we get $b=2$, $- 2c - b = 0 \Rightarrow c = -1$ and $-a + 2c + b = 3 \Rightarrow a = -3$. Hence, the solution is ($a = -3, b = 2, c = -1$)
```
````

````{exercise}  Finding solution(s)
:label: ex-2
Find the solution for the below set of equations using forward elimination and back substitution
\begin{eqnarray}
\label{ex2_sys}
x + 2y + 3z &= 1, \\
3x + 2y + z &= 1, \\
7x + 2y − 3z &= 1
\end{eqnarray}
:::{hint} Click me for a hint!
:class: dropdown
Can multiple values satisfy a set of equations?
:::
```{solution} ex-2
:class: dropdown
We first eliminate $x$ from the second and third equations by doing [second eqn] $\gets$ [second eqn] - 3[first eqn]  and [third eqn] $\gets$ [third eqn] - 7[first eqn] to get
\begin{eqnarray}
\label{ex2_soln1}
x + 2y + 3z &= 1, \\
 -4y - 8z &= -2, \\
 -12y − 24z &= -6
\end{eqnarray}
Next, we eliminate $y$ from the third equation by doing [third eqn] $\gets$ [third eqn] - 3[second eqn] and get
\begin{eqnarray}
\label{ex2_soln2}
x + 2y + 3z &= 1, \\
 -4y - 8z &= -2, \\
 0 &= 0
\end{eqnarray}
In [](#ex2_soln2), the third equation is just a universal fact ($0=0$!). The second equation $-4y - 8z = -2$ has infinitely many solutions. For example, if $y=0$, then $z = \frac{1}{4}$. Given some value for $y$, we can always find a value for $z$ as $z = \frac{1 - 2y}{4}$. Substituting the corresponding value of $(y, z)$ in the first equation, we can always find a value for $x$ as $x = 1 - 2y - 3z$. 
```
````

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/00_Ch_1_Linear_Algebraic_Systems/021-linsys-gauss.ipynb)
